In [57]:
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import os
from nbconvert import ScriptExporter

In [59]:
raw_path = './../data/raw/UCI_Credit_Card.csv'                   # Ноутбук
# raw_path = './../../data/raw/UCI_Credit_Card.csv'                # Скрипт
processed_path = './../data/processed/UCI_Credit_Card.csv'       # Ноутбук
# processed_path = './../../data/processed/UCI_Credit_Card.csv'    # Скрипт

In [60]:
# Создаем функцию для загрузки датасета
def load_data(data_path=raw_path):
    df = pd.read_csv(data_path)
    return df

In [61]:
# Посмотрим на наш датасет
df = load_data()
print(f'Размер таблицы: {df.shape}')
print('Первые строки:')
print(df.head())
print('Типы данных:')
print(df.dtypes)

Размер таблицы: (30000, 25)
Первые строки:
   ID  LIMIT_BAL  SEX  EDUCATION  MARRIAGE  AGE  PAY_0  PAY_2  PAY_3  PAY_4  \
0   1    20000.0    1          2         1   24      2      2     -1     -1   
1   2   120000.0    2          2         2   26     -1      2      0      0   
2   3    90000.0    2          2         2   34      0      0      0      0   
3   4    50000.0    2          2         1   37      0      0      0      0   
4   5    50000.0    1          2         1   57     -1      0     -1      0   

   ...  BILL_AMT4  BILL_AMT5  BILL_AMT6  PAY_AMT1  PAY_AMT2  PAY_AMT3  \
0  ...        0.0        0.0        0.0       0.0     689.0       0.0   
1  ...     3272.0     3455.0     3261.0       0.0    1000.0    1000.0   
2  ...    14331.0    14948.0    15549.0    1518.0    1500.0    1000.0   
3  ...    28314.0    28959.0    29547.0    2000.0    2019.0    1200.0   
4  ...    20940.0    19146.0    19131.0    2000.0   36681.0   10000.0   

   PAY_AMT4  PAY_AMT5  PAY_AMT6  default.pa

In [15]:
# Создадим функцию, которая будет производить разведывательный анализ датасета
def explore_data(df):
    # Находим количсество пропусков в данных
    print('Количество пропусков:')
    print(df.isna().sum())

    # Оцениваем процентное распределение целевой переменной
    target_distribution = df['default.payment.next.month'].value_counts(normalize=True)*100
    print('\nРаспределение целевой переменной (%):')
    print(target_distribution)

    # Выводим статистику по числовым признакам
    numeric_columns = df.select_dtypes(include=['float', 'int']).columns
    print('\nСтатистика по числовым признакам:')
    print(df[numeric_columns].describe())

    # Выведем значения категориальных признаков
    categorical_columns = df.select_dtypes(exclude=['float', 'int']).columns
    if len(categorical_columns) > 0:
        print('\nКатегоричные признаки:')
        for col in categorical_columns:
            print(f'{col}: {df[col].unique()}')

In [16]:
explore_data(df)

Количество пропусков:
ID                            0
LIMIT_BAL                     0
SEX                           0
EDUCATION                     0
MARRIAGE                      0
AGE                           0
PAY_0                         0
PAY_2                         0
PAY_3                         0
PAY_4                         0
PAY_5                         0
PAY_6                         0
BILL_AMT1                     0
BILL_AMT2                     0
BILL_AMT3                     0
BILL_AMT4                     0
BILL_AMT5                     0
BILL_AMT6                     0
PAY_AMT1                      0
PAY_AMT2                      0
PAY_AMT3                      0
PAY_AMT4                      0
PAY_AMT5                      0
PAY_AMT6                      0
default.payment.next.month    0
dtype: int64

Распределение целевой переменной (%):
0    77.88
1    22.12
Name: default.payment.next.month, dtype: float64

Статистика по числовым признакам:
            LIMIT_BA

In [17]:
# Создадим функцию, которая будет предобрабатывать данные
def preprocess_data(df):
    # Удаляем столбец ID
    df.drop(columns=["ID"], inplace=True)

    # Производим биннинг возраста
    bins = [20, 30, 40, 50, float('inf')]
    labels = ['Молодые', 'Средний возраст', 'Взрослые', 'Старшие']
    df['AGE_GROUP'] = pd.cut(df['AGE'], bins=bins, labels=labels)

    # Выполняем OHE-обработку категориальных признаков
    cat_cols = ['SEX', 'MARRIAGE', 'EDUCATION']
    df_encoded = pd.get_dummies(df, columns=cat_cols, drop_first=True)

    # Нормализуем числовые признаки
    scaler = MinMaxScaler()
    num_cols = ['LIMIT_BAL'] + [f'BILL_AMT{i}' for i in range(1, 7)] + [f'PAY_AMT{i}' for i in range(1, 7)]
    df_scaled = scaler.fit_transform(df_encoded[num_cols])
    df_encoded[num_cols] = df_scaled

    return df_encoded

In [26]:
# Обрабатываем данные и сохраняем датасет
df_processed = preprocess_data(df)
output_path = Path(processed_path+'preprocessed_data.csv')
df_processed.to_csv(output_path, index=False)
print(f'\nДанные успешно сохранены в {output_path}.')


Данные успешно сохранены в ..\data\processed\preprocessed_data.csv.


In [50]:
# Сохранение ноутбука, как скрипт
# notebook_name = 'import_and_eda.ipynb'
# output_folder = './../src/data/'

# exporter = ScriptExporter()
# script, resources = exporter.from_filename(notebook_name)
# script = script[:-430]

# os.makedirs(output_folder, exist_ok=True)
# with open(os.path.join(output_folder, notebook_name.replace('import_and_eda.ipynb', 'make-dataset.py')), 'w') as f:
#     f.write(script)
#     print('Скрипт сохранен')

Скрипт сохранен
